In [2]:
#data handling
import pandas as pd
import numpy as np
import glob, os

# Bokeh libraries
from bokeh.io import output_file, output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.layouts import row, column, gridplot
from bokeh.models import TabPanel
from bokeh.io import output_file
from bokeh.plotting import figure, show

In [4]:
# Get CSV files list from a folder
path = 'data/crashes_data'
csv_files = glob.glob(path + "/*.csv")

# Read each CSV file into DataFrame
# This creates a list of dataframes
df_list = (pd.read_csv(file, encoding = 'utf-16') for file in csv_files)


#encoding moest hier gedefinieerd worden anders kreeg ik een error, https://stackoverflow.com/questions/50342517/unicodedecodeerror-reading-a-csv-file

# Concatenate all DataFrames
df   = pd.concat(df_list, ignore_index=True)

print(df)

           Date                         Package Name  Daily Crashes  \
0    2021-06-01  com.vansteinengroentjes.apps.ddfive             15   
1    2021-06-02  com.vansteinengroentjes.apps.ddfive             12   
2    2021-06-03  com.vansteinengroentjes.apps.ddfive             20   
3    2021-06-04  com.vansteinengroentjes.apps.ddfive             13   
4    2021-06-05  com.vansteinengroentjes.apps.ddfive             14   
..          ...                                  ...            ...   
209  2021-12-27  com.vansteinengroentjes.apps.ddfive             64   
210  2021-12-28  com.vansteinengroentjes.apps.ddfive             60   
211  2021-12-29  com.vansteinengroentjes.apps.ddfive             37   
212  2021-12-30  com.vansteinengroentjes.apps.ddfive             46   
213  2021-12-31  com.vansteinengroentjes.apps.ddfive             50   

     Daily ANRs  
0             1  
1             1  
2             1  
3             0  
4             0  
..          ...  
209           0  
210

In [5]:
df = df[["Date","Package Name","Daily Crashes","Daily ANRs"]]
#selecteert de rijen die van belang zijn (zie opdracht document)

print(df)

           Date                         Package Name  Daily Crashes  \
0    2021-06-01  com.vansteinengroentjes.apps.ddfive             15   
1    2021-06-02  com.vansteinengroentjes.apps.ddfive             12   
2    2021-06-03  com.vansteinengroentjes.apps.ddfive             20   
3    2021-06-04  com.vansteinengroentjes.apps.ddfive             13   
4    2021-06-05  com.vansteinengroentjes.apps.ddfive             14   
..          ...                                  ...            ...   
209  2021-12-27  com.vansteinengroentjes.apps.ddfive             64   
210  2021-12-28  com.vansteinengroentjes.apps.ddfive             60   
211  2021-12-29  com.vansteinengroentjes.apps.ddfive             37   
212  2021-12-30  com.vansteinengroentjes.apps.ddfive             46   
213  2021-12-31  com.vansteinengroentjes.apps.ddfive             50   

     Daily ANRs  
0             1  
1             1  
2             1  
3             0  
4             0  
..          ...  
209           0  
210

In [6]:
df.isna().sum()/len(df)*100

#laat het percentage NaN values zijn.



Date             0.0
Package Name     0.0
Daily Crashes    0.0
Daily ANRs       0.0
dtype: float64

In [9]:
df['Date'] = pd.to_datetime(df['Date'])
#Vertaalt de data in de kolom 'date' naar leesbaar data voor de pandas dataframe  
df_crashes_ANRs = df

print(df_crashes_ANRs)

          Date                         Package Name  Daily Crashes  Daily ANRs
0   2021-06-01  com.vansteinengroentjes.apps.ddfive             15           1
1   2021-06-02  com.vansteinengroentjes.apps.ddfive             12           1
2   2021-06-03  com.vansteinengroentjes.apps.ddfive             20           1
3   2021-06-04  com.vansteinengroentjes.apps.ddfive             13           0
4   2021-06-05  com.vansteinengroentjes.apps.ddfive             14           0
..         ...                                  ...            ...         ...
209 2021-12-27  com.vansteinengroentjes.apps.ddfive             64           0
210 2021-12-28  com.vansteinengroentjes.apps.ddfive             60           0
211 2021-12-29  com.vansteinengroentjes.apps.ddfive             37           0
212 2021-12-30  com.vansteinengroentjes.apps.ddfive             46           1
213 2021-12-31  com.vansteinengroentjes.apps.ddfive             50           0

[214 rows x 4 columns]


In [11]:
output_notebook()

#selecteerd de kolommen 'date' en 'Daily Crashes'
df_date_crashes = df_crashes_ANRs[["Date", "Daily Crashes"]]

#Maakt cds bestanden
df_date_crashes_cds = ColumnDataSource(df_crashes_ANRs.groupby('Date')['Daily Crashes'].sum().to_frame().reset_index())

print(df_date_crashes)


Loading BokehJS ...

          Date  Daily Crashes
0   2021-06-01             15
1   2021-06-02             12
2   2021-06-03             20
3   2021-06-04             13
4   2021-06-05             14
..         ...            ...
209 2021-12-27             64
210 2021-12-28             60
211 2021-12-29             37
212 2021-12-30             46
213 2021-12-31             50

[214 rows x 2 columns]


In [12]:
df_date_avg_crashes_monthly = df_date_crashes.groupby(pd.Grouper(key='Date', freq='M')).mean().reset_index()

#dit geeft de gemiddelde average crashes per maand

print(df_date_avg_crashes_monthly)

        Date  Daily Crashes
0 2021-06-30      13.466667
1 2021-07-31      16.322581
2 2021-08-31      12.806452
3 2021-09-30      15.000000
4 2021-10-31      19.935484
5 2021-11-30      31.466667
6 2021-12-31      56.225806


In [14]:
from bokeh.models import HoverTool
from bokeh.models import DatetimeTickFormatter
output_notebook()

fig = figure(title = 'Average crashes per month', 
             x_axis_type='datetime', 
             height=400, 
             width=800, 
             x_axis_label='Date', 
             y_axis_label='Crashes')
#Zet het figuur op

source = ColumnDataSource(df_date_avg_crashes_monthly)

# Add the hover tool to the figure
hover = HoverTool(tooltips=[('Date', '@Date{%F}'), ('Crashes', '@{Daily Crashes}')],
                  formatters={'@Date': 'datetime'})

fig.vbar(x='Date', top='Daily Crashes', source=source, width=86400000*28)

# Add the hover tool to the figure
fig.add_tools(hover)


show(fig)

#x-as klopt nog niet, hij begint 1 maand later dan in de dataframe!


Loading BokehJS ...

In [15]:
output_notebook()

#selecteerd de kolommen 'date' en 'Daily ANRs'
df_date_ANRs = df_crashes_ANRs[["Date", "Daily ANRs"]]

#Maakt cds bestanden
df_date_ANRs_cds = ColumnDataSource(df_crashes_ANRs.groupby('Date')['Daily ANRs'].sum().to_frame().reset_index())

print(df_date_ANRs)

Loading BokehJS ...

          Date  Daily ANRs
0   2021-06-01           1
1   2021-06-02           1
2   2021-06-03           1
3   2021-06-04           0
4   2021-06-05           0
..         ...         ...
209 2021-12-27           0
210 2021-12-28           0
211 2021-12-29           0
212 2021-12-30           1
213 2021-12-31           0

[214 rows x 2 columns]


In [17]:
df_date_avg_ANRs_monthly = df_date_ANRs.groupby(pd.Grouper(key='Date', freq='M')).mean().reset_index()

#dit geeft de gemiddelde average ANRs per maand

print(df_date_avg_ANRs_monthly)

        Date  Daily ANRs
0 2021-06-30    0.400000
1 2021-07-31    1.000000
2 2021-08-31    0.322581
3 2021-09-30    0.533333
4 2021-10-31    0.419355
5 2021-11-30    0.400000
6 2021-12-31    0.129032


In [19]:
from bokeh.models import HoverTool
from bokeh.models import DatetimeTickFormatter
output_notebook()

fig = figure(title = 'Average ANRs per month', 
             x_axis_type='datetime', 
             height=400, 
             width=800, 
             x_axis_label='Date', 
             y_axis_label='ANRs')
#Zet het figuur op

source = ColumnDataSource(df_date_avg_ANRs_monthly)

# Add the hover tool to the figure
hover = HoverTool(tooltips=[('Date', '@Date{%F}'), ('ANRs', '@{Daily ANRs}')],
                  formatters={'@Date': 'datetime'})

fig.vbar(x='Date', top='Daily ANRs', source=source, width=86400000*28)

# Add the hover tool to the figure
fig.add_tools(hover)


show(fig)

#x-as klopt nog niet, hij begint 1 maand later dan in de dataframe!


Loading BokehJS ...